Dependencies
-------------

In [1]:
%AddDeps org.bdgenomics.adam adam-core-spark2_2.11 0.24.0
%AddDeps comp.bio.aging adam-playground_2.11 0.0.13 --repository https://dl.bintray.com/comp-bio-aging/main/

Marking org.bdgenomics.adam:adam-core-spark2_2.11:0.24.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir1840453984096215230/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir1840453984096215230/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.jar
-> New file at /tmp/toree-tmp-dir1840453984096215230/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.pom


Waiting for a Spark session to start...

Marking comp.bio.aging:adam-playground_2.11:0.0.13 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir1840453984096215230/toree_add_deps/
-> https://dl.bintray.com/comp-bio-aging/main/
-> https://repo1.maven.org/maven2


In [2]:
%AddDeps org.apache.hadoop hadoop-azure 2.7.6
%AddDeps com.microsoft.azure azure-storage 2.0.0

Marking org.apache.hadoop:hadoop-azure:2.7.6 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir1840453984096215230/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir1840453984096215230/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.jar
-> New file at /tmp/toree-tmp-dir1840453984096215230/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.pom


Waiting for a Spark session to start...

Marking com.microsoft.azure:azure-storage:2.0.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir1840453984096215230/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir1840453984096215230/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.jar
-> New file at /tmp/toree-tmp-dir1840453984096215230/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.pom


Imports and general functions
-----------------------------

In [3]:
import  org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types.StructType
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.ml.linalg._
import org.apache.spark.ml.stat._

In [4]:
def sparkHadoopConf(sc: SparkContext, acountName: String, accountKey: String) = {
  sc.hadoopConfiguration.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
  sc.hadoopConfiguration.set("fs.azure.account.key." + acountName + ".blob.core.windows.net", accountKey)
  sc
}

sparkHadoopConf: (sc: org.apache.spark.SparkContext, acountName: String, accountKey: String)org.apache.spark.SparkContext


In [5]:
def readTSV(path: String, header: Boolean = false, sep: String = "\t", infer: Boolean = true): DataFrame = spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("inferSchema", infer)
    .option("header", header)
    .option("maxColumns", 150000)
    .csv(path)

readTSV: (path: String, header: Boolean, sep: String, infer: Boolean)org.apache.spark.sql.DataFrame


In [6]:
def readTypedTSV[T <: Product](path: String, header: Boolean = false, sep: String = "\t")
                              (implicit tag: TypeTag[T]): Dataset[T] = {
  implicit val encoder: StructType = Encoders.product[T](tag).schema
  spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("header", header)
    .schema(encoder)
    .csv(path).as[T]
}

readTypedTSV: [T <: Product](path: String, header: Boolean, sep: String)(implicit tag: reflect.runtime.universe.TypeTag[T])org.apache.spark.sql.Dataset[T]


In [7]:
 def toVectors(dataFrame: DataFrame, columns: Seq[String], output: String) = {  
      import org.apache.spark.ml.feature.VectorAssembler
      import org.apache.spark.ml.linalg.Vectors

      val assembler = new VectorAssembler()
        .setInputCols(columns.toArray)
        .setOutputCol(output)

      assembler.transform(dataFrame.na.fill(0.0, columns).na.fill("")).select(output)
    }

toVectors: (dataFrame: org.apache.spark.sql.DataFrame, columns: Seq[String], output: String)org.apache.spark.sql.DataFrame


In [8]:
def azurize(container: String, accountName: String, blobFile: String): String = "wasbs://"+container+"@"+accountName+".blob.core.windows.net/"+blobFile 
val account = "pipelines1"
val key = ""
val connString = s"DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=${key};EndpointSuffix=core.windows.net"
def az(path: String): String = azurize("storage", account, path)

account = pipelines1
key = 
connString = DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net


azurize: (container: String, accountName: String, blobFile: String)String
az: (path: String)String


DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net

In [9]:
def write(df: DataFrame, url: String, header: Boolean = true) = {
  df.coalesce(1).write.option("sep","\t").option("header", header).csv(url)
  url
}

write: (df: org.apache.spark.sql.DataFrame, url: String, header: Boolean)String


Core code
---------
Loading pathes
--------------

In [10]:
val sparkContext = sc
sparkHadoopConf(sparkContext, account, key)
  
val spark = SparkSession
  .builder()
  .appName("whales_validation")
  .getOrCreate()

sparkContext = org.apache.spark.SparkContext@1a66e811
spark = org.apache.spark.sql.SparkSession@2c3cecb7


Spark app-20180704154408-0007: Some(http://c8fc1f462c1d:4040)

In [11]:
val expressionsPath = "expressions"
val byGoPath = expressionsPath + "/go"
val comparison = byGoPath + "/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv"
val grouped = byGoPath + "/grouped"
val ranked = byGoPath + "/grouped/ranked"
val transcriptsPath = expressionsPath + "/transcripts"
val indexesPath = "indexes"
val reactomePath = indexesPath + "/reactome"
val unirefPath = expressionsPath + "/" + "uniref90"

expressionsPath = expressions
byGoPath = expressions/go
comparison = expressions/go/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv
grouped = expressions/go/grouped
ranked = expressions/go/grouped/ranked
transcriptsPath = expressions/transcripts
indexesPath = indexes
reactomePath = indexes/reactome
unirefPath = expressions/uniref90


expressions/uniref90

In [12]:
val goTable = readTSV(az(comparison), true)

goTable = [go: string, namespace: string ... 27 more fields]


error: error while loading QualifiedTableName, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/QualifiedTableName.class)' has location not matching its contents: contains class QualifiedTableName
error: error while loading JavaTypeInference, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/JavaTypeInference.class)' has location not matching its contents: contains class JavaTypeInference
error: error while loading FunctionIdentifier, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/FunctionIdentifier.class)' has location not matching its contents: contains class FunctionIdentifier
error: error while loading DefinedByConstructorParams, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/DefinedByConstructorParams.class)' has location 

[go: string, namespace: string ... 27 more fields]

In [13]:
%%dataframe goTable

error: error while loading Cross, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/Cross.class)' has location not matching its contents: contains class Cross
error: error while loading NaturalJoin, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/NaturalJoin.class)' has location not matching its contents: contains class NaturalJoin
error: error while loading LeftExistence, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/LeftExistence.class)' has location not matching its contents: contains class LeftExistence
error: error while loading RightOuter, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/RightOuter.class)' has location not matching its contents: contains class RightOuter
error: error while loading 

go,namespace,label,gray_whale_avg,uniref90_count,taxons_count,transcripts_count,human_transcripts_count,mouse_transcripts_count,cow_transcripts_count,type,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
GO:0070062,cellular_component,extracellular exosome,95231.76295,726,929,876,2714,116,2162,complete;5prime_partial;internal;3prime_partial,82627.97828900004,43406.81879601012,103879.82613499998,189153.98861699994,34179.85539700001,63059.86755299999,12146.704848999996,407115.5446490002,402875.4094160001,107835.54761100012,12291.983165,139226.62839100015,251177.67484400002,17317.253403999995,8189.928592999998,3009.2303389999997,284041.33320799994,274118.5267729999
GO:0016021,cellular_component,integral component of membrane,81241.0623675,1391,565,1713,9561,8900,4715,5prime_partial;complete;3prime_partial;internal,62374.28512099997,27353.04877027518,55178.75498600001,108836.22230300008,11032.347553,168821.304916,28909.156348999997,198371.61491800006,189068.7378029999,100107.839614,24343.312661999982,94079.22781100006,149277.02259700003,30508.606157000006,241562.48318399998,51496.39662599999,315467.63018200017,373243.68564099993
GO:0003735,molecular_function,structural constituent of ribosome,61936.161357499994,98,380,107,467,306,261,complete;5prime_partial;3prime_partial;internal,62265.27571699998,21665.888836018064,62369.073224,103095.90135300001,5244.261549,24249.30922799999,7137.034891999996,89084.343089,87166.34631000001,61607.046997999976,8349.420524000001,88459.54913499996,70943.44369100004,4565.115427999999,50372.43014700001,11549.923136999998,94610.67463199992,72399.43132700001
GO:0005829,cellular_component,cytosol,61398.1790675,1109,994,1376,6775,5043,2348,complete;3prime_partial;5prime_partial;internal,50912.655819000014,29665.787975965563,72109.60757099999,188757.6132710001,20623.12592999999,148373.97883200005,100424.52617,86184.9085600001,90951.61626199998,71883.70231600002,12335.804511999993,124936.20551199997,132241.5113150001,13435.613476000004,105758.17169699995,43636.518504000014,111689.96638100002,102744.94863500002
GO:0005840,cellular_component,ribosome,59087.688684,89,316,99,297,123,40,complete;5prime_partial;internal;3prime_partial,58631.94472899998,23801.48132475636,65881.81530099995,103165.58999999998,2209.9476119999995,5478.616020999998,1831.119411,5936.298134999999,6073.805264,59543.432639000006,9270.050664999997,92530.64897999998,71492.44906799999,1924.4887119999999,10029.277271,2800.6401849999993,5875.225427000001,4481.681578999999
GO:0006412,biological_process,translation,58622.12868799999,105,377,116,468,286,193,complete;5prime_partial;3prime_partial;internal,58727.85268499997,20034.802301962056,58984.534049,99321.06117499995,5245.888943999999,19875.159683999995,5806.578341,66453.233819,66083.20671399997,58516.40469099998,7580.873805999998,83876.08076099999,68055.43613499998,4572.8019749999985,40136.835409,9191.198294,69545.42429499998,53565.293888999986
GO:0005634,cellular_component,nucleus,57345.87068550001,1431,2115,1830,7301,6496,2778,complete;5prime_partial;3prime_partial;internal,55747.40112099999,30295.434521280793,70441.20552599998,140240.092983,10470.131235999997,94231.85965399996,84031.996854,126193.50819700002,143407.437234,58944.34024999998,11416.312302999997,102915.768411,90533.17986400002,8808.929934000003,81393.57837199996,30863.47935199999,118101.15175700001,99420.20529599997
GO:0003723,molecular_function,RNA binding,54252.262738,621,368,821,2148,1069,1008,complete;5prime_partial;3prime_partial;internal,48843.76697,23928.740201085533,56334.43345600005,91851.60494800007,6522.695184999994,14026.664751000004,4708.064785000002,74843.61990199998,82199.186156,59660.75850600002,10025.436343999996,93364.80089000006,79318.97048099998,6672.883201000003,24

In [14]:
goTable.count

18597

In [15]:
val allUniref = readTSV(az(unirefPath + "/all_uniref90_outer.tab"  ), true)

allUniref = [uniref90: string, gray_whale_liver: double ... 17 more fields]


[uniref90: string, gray_whale_liver: double ... 17 more fields]

In [16]:
%%dataframe allUniref

uniref90,gray_whale_liver,bowhead_whale_liver,minke_liver,NMR_liver,human_liver,mouse_totalRNA_liver,mouse_mRNA_liver,cow_totalRNA_liver,cow_mRNA_liver,gray_whale_kidney,bowhead_whale_kidney,minke_kidney,NMR_kidney,human_kidney,mouse_totalRNA_kidney,mouse_mRNA_kidney,cow_totalRNA_kidney,cow_mRNA_kidney
UniRef90_A0A1D3JP32,28912.07396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80702.3886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_B0JE22,23345.653565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44961.569793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_P0DMM1,13273.280865,2400.85,14073.428271,0.0,0.0,0.0,0.0,0.0,0.0,5.871845,25.693117,1321.29886,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_Q8IY31,10851.276487,0.0,31.497888,0.0,0.0,0.0,0.0,0.0,0.0,27763.924099,0.0,47.342378,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_Q95MP7,10473.906031,0.0,3043.48281,0.0,0.0,0.0,0.0,0.0,0.0,3570.825838,113.243136,2407.273068,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A286ZQC7,10060.120714,8386.28,24576.018837,0.0,0.0,0.0,0.0,0.0,0.0,1723.115534,46.086086,639.28565,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A212CAV9,6413.629848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12812.097645,1.358659,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A0E2JFH7,5342.438348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12792.727827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_L5MA95,3971.232224,871.505,3710.557181,0.0,0.0,0.0,0.0,0.0,0.0,3177.662877,67.772318,4141.129523,0.0,0.0,0.0,0.0,0.0,0.0
UniRef90_A0A140GN66,3903.081981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1555.815324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
allUniref.count

119429

In [18]:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.6.jar

Starting download from https://brunelvis.org/jar/spark-kernel-brunel-all-2.6.jar
Finished download of spark-kernel-brunel-all-2.6.jar


In [19]:
goTable.columns.drop(11)

[gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney]

In [36]:
allUniref.columns

[uniref90, gray_whale_liver, bowhead_whale_liver, minke_liver, NMR_liver, human_liver, mouse_totalRNA_liver, mouse_mRNA_liver, cow_totalRNA_liver, cow_mRNA_liver, gray_whale_kidney, bowhead_whale_kidney, minke_kidney, NMR_kidney, human_kidney, mouse_totalRNA_kidney, mouse_mRNA_kidney, cow_totalRNA_kidney, cow_mRNA_kidney]

In [37]:
val by_g = allUniref.select("uniref90","gray_whale_liver").sort($"gray_whale_liver".desc)

by_g = [uniref90: string, gray_whale_liver: double]


[uniref90: string, gray_whale_liver: double]

In [60]:
val by_minke = allUniref.select("uniref90","minke_kidney").sort($"minke_kidney".desc)

by_minke = [uniref90: string, minke_kidney: double]


lastException: Throwable = null


[uniref90: string, minke_kidney: double]

In [66]:
val by_bowhead = allUniref.select("uniref90","bowhead_whale_kidney").sort($"bowhead_whale_kidney".desc)

by_bowhead = [uniref90: string, bowhead_whale_kidney: double]


[uniref90: string, bowhead_whale_kidney: double]

In [68]:
%%brunel data('by_bowhead') top(bowhead_whale_kidney:3000) bar y(bowhead_whale_kidney:linear) x(#row) bin(bowhead_whale_kidney) axes(x, y)

                'brunel' : 'https://brunelvis.org/js...


In [58]:
%%brunel data('allUniref') sort(gray_whale_kidney) top(gray_whale_kidney:1000) bar y(gray_whale_kidney:linear) x(#row) axes(x, y)

                'brunel' : 'https://brunelvis.org/js...


In [45]:
%%brunel data('allUniref') bar x(gray_whale_liver) y(#count:linear) bin(10) style("size:100%")

Magic brunel failed to execute with error: 
bin does not have numeric parameters: 10 while parsing action text: data('allUniref') bar x(gray_whale_liver) y(#count:linear) bin(10) style("size:100%")

Histograms
==========

Uniref90 histograms
-------------------
Y axes is a number of Uniref90-s while X ages is TPMs, scale is logarifmic

Liver
-----

In [20]:
%%brunel data('allUniref') bar x(gray_whale_liver) y(#count) bin(gray_whale_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/a...


In [21]:
%%brunel data('allUniref') bar x(bowhead_whale_liver) y(#count) bin(bowhead_whale_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/a...


In [22]:
%%brunel data('allUniref') bar x(minke_liver) y(#count) bin(minke_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/a...


In [23]:
%%brunel data('allUniref') bar x(NMR_liver) y(#count) bin(NMR_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/a...


In [24]:
%%brunel data('allUniref') bar x(human_liver) y(#count) bin(human_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/a...


In [25]:
%%brunel data('allUniref') bar x(mouse_totalRNA_liver) y(#count) bin(mouse_totalRNA_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/a...


In [26]:
%%brunel data('allUniref') bar x(mouse_mRNA_liver) y(#count) bin(mouse_mRNA_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/a...


In [27]:
%%brunel data('allUniref') bar x(cow_totalRNA_liver) y(#count) bin(cow_totalRNA_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/a...


In [28]:
%%brunel data('allUniref') bar x(cow_mRNA_liver) y(#count) bin(cow_mRNA_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/a...


Kidney
------

In [29]:
%%brunel data('allUniref') bar x(gray_whale_kidney) y(#count) bin(gray_whale_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [30]:
%%brunel data('allUniref') bar x(bowhead_whale_kidney) y(#count) bin(bowhead_whale_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [31]:
%%brunel data('allUniref') bar x(minke_kidney) y(#count) bin(minke_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [32]:
%%brunel data('allUniref') bar x(NMR_kidney) y(#count) bin(NMR_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [33]:
%%brunel data('allUniref') bar x(human_kidney) y(#count) bin(human_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [34]:
%%brunel data('allUniref') top(mouse_totalRNA_kidney:1000) bar x(mouse_totalRNA_kidney:linear) y(#count:linear) bin(mouse_totalRNA_kidney:10) axes(x:grid, y:grid) style("size:200%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [35]:
%%brunel data('allUniref') bar x(mouse_mRNA_kidney) y(#count) bin(mouse_mRNA_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [36]:
%%brunel data('allUniref') bar x(cow_totalRNA_kidney) y(#count) bin(cow_totalRNA_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [37]:
%%brunel data('allUniref') bar x(cow_mRNA_kidney) y(#count) bin(cow_mRNA_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


GO histograms
----------------------

Y axes is a number of GO-s while X ages is TPMs, scale is logarifmic

Liver
-------

In [11]:
%%brunel data('goTable') bar x(gray_whale_liver) y(#count) bin(gray_whale_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [29]:
%%brunel data('goTable') bar x(bowhead_whale_liver) y(#count) bin(bowhead_whale_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [12]:
%%brunel data('goTable') bar x(minke_liver) y(#count) bin(minke_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [14]:
%%brunel data('goTable') bar x(NMR_liver) y(#count) bin(NMR_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [16]:
%%brunel data('goTable') bar x(human_liver) y(#count) bin(human_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [18]:
%%brunel data('goTable') bar x(mouse_totalRNA_liver) y(#count) bin(mouse_totalRNA_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [19]:
%%brunel data('goTable') bar x(mouse_mRNA_liver) y(#count) bin(mouse_mRNA_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [21]:
%%brunel data('goTable') bar x(cow_totalRNA_liver) y(#count) bin(cow_totalRNA_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [22]:
%%brunel data('goTable') bar x(cow_mRNA_liver) y(#count) bin(cow_mRNA_liver) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


Kidney, GO
----------------

In [17]:
%%brunel data('goTable') bar x(gray_whale_kidney) y(#count) bin(gray_whale_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [3]:
%%brunel data('goTable') bar x(bowhead_whale_kidney) y(#count) bin(bowhead_whale_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [4]:
%%brunel data('goTable') bar x(minke_kidney) y(#count) bin(minke_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [5]:
%%brunel data('goTable') bar x(NMR_kidney) y(#count) bin(NMR_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [7]:
%%brunel data('goTable') bar x(human_kidney) y(#count) bin(human_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [9]:
%%brunel data('goTable') bar x(mouse_totalRNA_kidney) y(#count) bin(mouse_totalRNA_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [10]:
%%brunel data('goTable') bar x(mouse_mRNA_kidney) y(#count) bin(mouse_mRNA_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...


In [ ]:
%%brunel data('goTable') bar x(cow_totalRNA_kidney) y(#count) bin(cow_mRNA_kidney) style("size:100%")

In [11]:
%%brunel data('goTable') bar x(cow_mRNA_kidney) y(#count) bin(cow_mRNA_kidney) style("size:100%")

                'topojson' : '//cdnjs.cloudflare.com/...
